In [ ]:
import tensorflow_datasets as tfds # tf data
import tensorflow as tf 
import matplotlib.pyplot as plt

In [ ]:
# tfds 데이터 다운 받기
dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True,
                          as_supervised=True
                          )
train_dataset, test_dataset = dataset['train'], dataset["test"]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteMLRIJW/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteMLRIJW/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [ ]:
encoder = info.features["text"].encoder

In [ ]:
# encoder test
sample_string = "Hello Tensorflow"

encoded_string = encoder.encode(sample_string)

In [ ]:
encoded_string

[4025, 222, 6307, 2327, 2934]

In [ ]:
encoder.decode([4025, 222, 6307, 2327, 2934])

'Hello Tensorflow'

In [ ]:
for index in [4025, 222, 6307, 2327, 2934]:
    print(index, "-->", encoder.decode([index]))


4025 --> Hell
222 --> o 
6307 --> Ten
2327 --> sor
2934 --> flow


In [ ]:
for x,y in train_dataset.take(1):
    print(x)
    print(y)

tf.Tensor(
[[  62   18    4 ...    0    0    0]
 [ 597   33   20 ...    0    0    0]
 [  12   56  245 ...    0    0    0]
 ...
 [7963 5263 4371 ...    0    0    0]
 [4756 5811  194 ...    0    0    0]
 [  12   80    1 ...    0    0    0]], shape=(128, 1414), dtype=int64)
tf.Tensor(
[0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1
 0 1 1 0 1 0 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1
 0 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1
 1 1 0 1 1 1 0 1 0 0 0 1 0 0 0 1 0], shape=(128,), dtype=int64)


In [ ]:
# 하이퍼 파라미터
BUFFER_SIZE = 10000
BATCH_SIZE = 128

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
# padded_batch -> 문자열중에 가장 긴 문장으로 나머지 0을 채움
train_dataset = train_dataset.padded_batch(BATCH_SIZE)

In [ ]:
test_dataset = test_dataset.padded_batch(BATCH_SIZE)

In [ ]:
from tensorflow import keras

In [ ]:
# 영화 리뷰 긍부정 예측
# x -> 영화리뷰
# y -> 리뷰에대한 긍부정 0 or 1

In [ ]:
model = keras.Sequential([
        keras.layers.Embedding(encoder.vocab_size, 64),
        keras.layers.Bidirectional(keras.layers.LSTM(64)),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(
    loss = "binary_crossentropy",
    optimizer = keras.optimizers.Adam(0.0001),
    metrics = ["accuracy"]
    )

In [ ]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset,
    validation_steps=30
)

Epoch 1/10
196/196 [==============================] - 81s 368ms/step - loss: 0.6925 - accuracy: 0.5283 - val_loss: 0.6890 - val_accuracy: 0.6151
Epoch 2/10
196/196 [==============================] - 72s 364ms/step - loss: 0.4884 - accuracy: 0.7805 - val_loss: 0.3883 - val_accuracy: 0.8404
Epoch 3/10
196/196 [==============================] - 72s 362ms/step - loss: 0.3052 - accuracy: 0.8825 - val_loss: 0.3428 - val_accuracy: 0.8591
Epoch 4/10
196/196 [==============================] - 72s 362ms/step - loss: 0.2453 - accuracy: 0.9106 - val_loss: 0.3379 - val_accuracy: 0.8703
Epoch 5/10
196/196 [==============================] - 71s 359ms/step - loss: 0.2117 - accuracy: 0.9255 - val_loss: 0.3398 - val_accuracy: 0.8724
Epoch 6/10
196/196 [==============================] - 71s 361ms/step - loss: 0.1885 - accuracy: 0.9349 - val_loss: 0.3375 - val_accuracy: 0.8750
Epoch 7/10
196/196 [==============================] - 71s 360ms/step - loss: 0.1712 - accuracy: 0.9420 - val_loss: 0.3458 - val_ac

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

196/196 [==============================] - 24s 124ms/step - loss: 0.3803 - accuracy: 0.8610


In [ ]:
def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec
def sample_predict(sample_pred_text, pad):
    encoded_sample_pred_text = encoder.encode(sample_pred_text)

    if pad:
        encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
    encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
    predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

    return (predictions)

In [ ]:
sample_pred_text = ("The movie was very good.")
prediction = sample_predict(sample_pred_text, pad=True)
print(prediction)

[[0.5675846]]
